In [260]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
import lxml
import requests
import unicodedata

In [261]:
fm_df = pd.read_csv("uluta_fm23data.csv")
# platinum_df = pd.read_csv("platinum22_fm23data.csv")

In [262]:
fm_df.columns = [col.strip() for col in fm_df.columns]

for col in fm_df.select_dtypes("object").columns:
   fm_df[col] = fm_df[col].apply(lambda x: x.strip())

In [263]:
convert_date_format = lambda x: pd.to_datetime(x, format='%d/%m/%Y').strftime('%Y-%m-%d')
fm_df['DoB'] = fm_df['DoB'].apply(lambda x: x.split(" ")[0])
fm_df['DoB'] = fm_df['DoB'].apply(convert_date_format)

In [264]:
fm_df['Height'] = fm_df['Height'].apply(lambda x: x.split(" ")[0])

In [265]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best Pos', 'Acc', 'Aer', 'Agg', 'Agi', 'Ant', 'Bal', 'Bra', 'Cmd',
       'Com', 'Cmp', 'Cnt', 'Cor', 'Cro', 'Dec', 'Det', 'Dri', 'Ecc', 'Fin',
       'Fir', 'Fla', 'Fre', 'Han', 'Hea', 'Jum', 'Kic', 'Ldr', 'Lon', 'L Th',
       'Mar', 'Nat .1', 'OtB', '1v1', 'Pac', 'Pas', 'Pen', 'Pos', 'Pun', 'Ref',
       'TRO', 'Sta', 'Str', 'Tck', 'Tea', 'Tec', 'Thr', 'Vis', 'Wor'],
      dtype='object')

In [266]:
fm_df.loc[323].to_dict()

{'UID': 14023219,
 'Inf': '',
 'Name': 'Guillermo Fernández',
 'DoB': '1991-10-11',
 'Nat': 'ARG',
 'Division': 'Argentine Premier Division',
 'Club': 'Boca',
 'Based': 'Argentina (Premier Division)',
 'Preferred Foot': 'Right',
 'Right Foot': 'Very Strong',
 'Left Foot': 'Reasonable',
 'Position': 'DM, M/AM (C)',
 'Height': '178',
 'Weight': '75 kg',
 'Age': 30,
 'Transfer Value': '€950K - €5.4M',
 'Wage': '€68,500 p/m',
 'AT Apps': '240',
 'AT Gls': '22',
 'Team': '',
 'Caps': 0,
 'Yth Apps': '-',
 'Style': 'Intelligent',
 'Rc Injury': '-',
 'Best Role': 'Deep Lying Playmaker',
 'Best Duty': 'Support',
 'Best Pos': 'M (C)',
 'Acc': 12,
 'Aer': 2,
 'Agg': 9,
 'Agi': 13,
 'Ant': 14,
 'Bal': 12,
 'Bra': 9,
 'Cmd': 2,
 'Com': 3,
 'Cmp': 12,
 'Cnt': 14,
 'Cor': 13,
 'Cro': 13,
 'Dec': 14,
 'Det': 13,
 'Dri': 13,
 'Ecc': 4,
 'Fin': 12,
 'Fir': 14,
 'Fla': 13,
 'Fre': 12,
 'Han': 3,
 'Hea': 7,
 'Jum': 9,
 'Kic': 1,
 'Ldr': 6,
 'Lon': 12,
 'L Th': 5,
 'Mar': 9,
 'Nat .1': 15,
 'OtB': 11,
 '1

<br><br><br><br>
# Merged_DF

In [267]:
selected_leagues = ['English Premier Division',
 'Ligue 1 Uber Eats',
 'Spanish First Division',
 'Bundesliga',
 'Italian Serie A',
 'Eredivisie',
 'Portuguese Premier League',
 'Turkish Super League']

In [268]:
fm_df = fm_df[fm_df.Division.isin(selected_leagues)]

In [269]:
for leg in selected_leagues:
    q = fm_df.query(f"Division == '{leg}'")["Club"].value_counts()
    print(leg , "------", len(q.to_list()))

English Premier Division ------ 20
Ligue 1 Uber Eats ------ 20
Spanish First Division ------ 20
Bundesliga ------ 18
Italian Serie A ------ 20
Eredivisie ------ 18
Portuguese Premier League ------ 18
Turkish Super League ------ 19


In [270]:
fm_df.query(f"Division == 'Turkish Super League'")["Club",""].value_counts()

KeyError: ('Club', '')

In [ ]:
fm_df.query(f"Club == 'Adana D.S.'")

<br><br><br><br>
# Team Names Converter from fminside.com

In [ ]:
url_df = fm_df.drop_duplicates(subset=["Club"], keep="first")
url_df.loc[:, 'UID'] = url_df['UID'].apply(lambda text: str(text))
url_df = url_df[["Club", "UID"]]

In [ ]:
def Player_to_URL(player_id):
    slug = player_id + "-x-x"
    return f"https://fminside.net/players/3-fm-23/{slug}"

In [ ]:
def Convert_Team_Name_From_URL(player_id, club):
    url = Player_to_URL(player_id)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")
    club_name_element = soup.find('div', class_='meta').find('ul').find_all('li')[0].text
    club_name = club_name_element.strip()
    return {"fm":club,"normal":club_name}

In [ ]:
team_mapping_array = []
raw_names = url_df.to_dict()
uids = url_df["UID"]
clubs = url_df["Club"]
clubs_array = clubs.to_list()

In [ ]:
# for uid,club in zip(uids,clubs):
#     new_entry = Convert_Team_Name_From_URL(uid, club)
#     print(new_entry)
#     team_mapping_array.append(new_entry)

In [ ]:
fminside_normal_names = pd.read_csv("fminside_team_names.csv")
merged_df = pd.merge(fm_df, fminside_normal_names, how='inner', left_on='Club', right_on='fm')
merged_df['Club'] = merged_df['normal'].where(merged_df['Club'] == merged_df['fm'], merged_df['Club'])
# merged_df.drop(['fm', 'normal'], axis=1, inplace=True)
merged_df = merged_df[merged_df.fm.isin(clubs_array)]

In [ ]:
merged_df.query("Name in 'Gökhan Inler'").to_dict()